In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xgcm
import xmitgcm as xm
import cftime

In [2]:
path = "/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_fine_3y/Diags"

In [3]:
ds = xm.open_mdsdataset(
    data_dir="/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_fine_3y/Diags",
    grid_dir="/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_fine_3y",
    prefix=["state", "2D_diags"],
    read_grid=True,
    geometry="cartesian",
    delta_t = 250,
    ref_date = '2000-1-1 0:0:0',
    calendar = '360_day',
).isel(time=slice(0,360*2))

In [4]:
ds_coarse = xm.open_mdsdataset(
    data_dir="/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_coarse_5y/Diags",
    grid_dir="/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_coarse_5y",
    prefix=["state", "2D_diags"],
    read_grid=True,
    geometry="cartesian",
    delta_t = 1000,
    ref_date = '2000-1-1 12:0:0',
    calendar = '360_day',
).isel(time=slice(0,360*2))

In [6]:
grid = xgcm.Grid(ds, periodic=['X'], boundary='extend', metrics={
        ('X',): ['dxC', 'dxG','dxF', 'dxV'], # X distances
        ('Y',): ['dyC', 'dyG','dyF', 'dyU'], # Y distances
        ('Z',): ['drC', 'drF'], # Z distances
        ('X', 'Y'): ['rA', 'rAz', 'rAs', 'rAw'] # Areas
    })

In [7]:
coarse_grid = xgcm.Grid(ds_coarse, periodic=['X'], boundary='extend', metrics={
        ('X',): ['dxC', 'dxG','dxF', 'dxV'], # X distances
        ('Y',): ['dyC', 'dyG','dyF', 'dyU'], # Y distances
        ('Z',): ['drC', 'drF'], # Z distances
        ('X', 'Y'): ['rA', 'rAz', 'rAs', 'rAw'] # Areas
    })

# Create a coarsened dataset

In [38]:
def coarsen(ds, ds_coarse, grid=True, kpp=False, debugW=False):
    attributes = {}
    for var in (ds_coarse.variables):
        attributes[var] = ds_coarse[var].attrs
        
    coarse_grid = xgcm.Grid(ds_coarse, periodic=['X'], boundary='extend', metrics={
        ('X',): ['dxC', 'dxG','dxF', 'dxV'], # X distances
        ('Y',): ['dyC', 'dyG','dyF', 'dyU'], # Y distances
        ('Z',): ['drC', 'drF'], # Z distances
        ('X', 'Y'): ['rA', 'rAz', 'rAs', 'rAw'] # Areas
    })
    
    ds_coarsened = xr.Dataset()

    print("Coarsening `ETAN`     ", end='\r')
    ds_coarsened['ETAN'] = ds.ETAN.coarsen(dim={'XC': 10, 'YC': 10}).mean().compute()
    print("Coarsening `MXLDEPTH`     ", end='\r')
    ds_coarsened['MXLDEPTH'] = ds.MXLDEPTH.coarsen(dim={'XC': 10, 'YC': 10}).mean().compute()
    print("Coarsening `SALT`      ", end='\r')
    SALTcontent = (ds.SALT*ds.drF * ds.rA * ds.hFacC).coarsen(dim={'XC': 10, 'YC': 10}).sum().compute()
    newSALT = (SALTcontent/(ds_coarse.drF * ds_coarse.rA * ds_coarse.hFacC)).compute().fillna(0)
    ds_coarsened['SALT'] = newSALT.where(np.isfinite(newSALT), 0).compute()
    print("Coarsening `THETA`  ", end='\r')
    THETAcontent = (ds.THETA*ds.drF * ds.rA * ds.hFacC).coarsen(dim={'XC': 10, 'YC': 10}).sum().compute()
    newTHETA = (THETAcontent/(ds_coarse.drF * ds_coarse.rA * ds_coarse.hFacC)).compute().fillna(0)
    ds_coarsened['THETA'] = newTHETA.where(np.isfinite(newTHETA), 0).compute()
    if kpp == True:
        variables.append("KPPhbl")
        variables.append("KPPdiffS")
        print("Coarsening `KPPhbl`  ", end='\r')
        ds_coarsened['KPPhbl'] = ds.KPPhbl.coarsen(dim={'XC': 10, 'YC': 10}).mean().compute()
        print("Coarsening `KPPdiffS`  ", end='\r')
        ds_coarsened['KPPdiffS'] = ds.KPPdiffS.coarsen(dim={'XC': 10, 'YC': 10}).mean().compute()
    for var in (ds_coarse.variables):
        ds_coarse[var].attrs = attributes[var]
    print("Coarsening `UVEL`      ", end='\r')
    u_transport = ds.UVEL * ds.dyG * ds.hFacW * ds.drF
    u_transport_coarse = u_transport[:, :, :, ::10].coarsen(dim={'YC': 10}).sum()
    newUVEL = (u_transport_coarse / (ds_coarse.dyG * ds_coarse.hFacW * ds_coarse.drF)).fillna(0)
    ds_coarsened['UVEL'] = newUVEL.where(np.isfinite(newUVEL), 0).compute()
    print("Coarsening `VVEL`      ", end='\r')
    v_transport = ds.VVEL * ds.dxG * ds.hFacS * ds.drF
    v_transport_coarse = v_transport[:, :, ::10, :].coarsen(dim={'XC': 10}).sum()
    newVVEL = (v_transport_coarse / (ds_coarse.dxG * ds_coarse.hFacS * ds_coarse.drF)).fillna(0)
    ds_coarsened['VVEL'] = newVVEL.where(np.isfinite(newVVEL), 0).compute()
    print("Computing `WVEL` from non-divergence", end='\r')
    wvel_coarsened = ds.WVEL.coarsen(dim={'XC': 10, 'YC': 10}).mean()
    ds_coarsened['WVEL'] = (coarse_grid.cumsum(coarse_grid.diff(u_transport_coarse, 'X') + coarse_grid.diff(v_transport_coarse, 'Y', boundary='fill'), axis='Z', boundary='fill')/ds_coarse.rA + wvel_coarsened.isel(Zl=0)).compute()

    for coord in list(ds_coarse.coords):
        if coord not in list(ds_coarsened):
            ds_coarsened[coord] = ds_coarse[coord]
            ds_coarsened = ds_coarsened.set_coords(coord)

    for var in list(ds_coarsened.variables):
        ds_coarsened[var].attrs = attributes[var]        
    
    print("Done                                 ", end='\r')
    
    
    returnVars = [ds_coarsened]
    if grid:
        returnVars.append(coarse_grid)
    if debugW:
        returnVars.append(wvel_coarsened)
    
    if len(returnVars) == 1:
        return ds_coarsened
    else:
        return returnVars

In [39]:
ds_coarsened, coarse_grid, wvel_coarsened = coarsen(ds, ds_coarse, debugW=True)

/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/daanr/miniconda3/envs/oceanography/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [51]:
ds_coarsened.to_netcdf(path="/projects/0/topios/hydrodynamic_data/MITgcm/ACC_channel/ACC_ridge_fine_2y_coarsened.nc")